In [0]:
import requests
import time
from config.aerodemo_config import get_config  # Import the centralized config

# Retrieve configuration for the current environment
env = dbutils.widgets.get("ENV")
config = get_config(env)

# ---------- CONFIG ----------
DATABRICKS_INSTANCE = config['databricks_instance']
TOKEN = config['pat_token']
JOB_ID = config['e2e_workflow_job_id']
# ----------------------------

headers = {
    "Authorization": f"Bearer {TOKEN}",
    "Content-Type": "application/json"
}

# ✅ Trigger the job run
run_url = f"{DATABRICKS_INSTANCE}/api/2.1/jobs/run-now"
payload = {"job_id": JOB_ID}

run_response = requests.post(run_url, headers=headers, json=payload)
if run_response.status_code != 200:
    print(f"❌ Failed to trigger job: {run_response.text}")
    raise SystemExit

run_data = run_response.json()
run_id = run_data["run_id"]
print(f"✅ Triggered job run. Run ID: {run_id}")

# ✅ Poll for run status
status_url = f"{DATABRICKS_INSTANCE}/api/2.1/jobs/runs/get?run_id={run_id}"

while True:
    status_response = requests.get(status_url, headers=headers)
    if status_response.status_code != 200:
        print(f"❌ Failed to get run status: {status_response.text}")
        break

    status_data = status_response.json()
    life_cycle_state = status_data["state"]["life_cycle_state"]
    result_state = status_data["state"].get("result_state")

    print(f"🔄 Run status: {life_cycle_state} (Result: {result_state})")

    if life_cycle_state in ["TERMINATED", "SKIPPED", "INTERNAL_ERROR"]:
        print(f"✅ Final state: {result_state}")
        break

    time.sleep(30)  # Poll every 30 seconds